# GNSS

References: 
- `Linear Algebra, Geodesy, and GPS`
- `https://www.geometh-data.ethz.ch/student/anleitungen/leicaman/diverses/GPSBasics_en.pdf`

## Receiver Position from Code Observations 

Assume the receiver clock bias is $t_{b_r}$ and the signal receiving time is $t_r$ and sending time is $t_s$, the pseudo range $ \rho = c * (t_r + t_{b_r} - t_s)$. If $t_{b_r}$ is positive, this means that the receiver's clock is slower than it should be. The receiver perceives the signal arrives earlier than it actually is. So the pseudo range is smaller than it actually is. This is why we will have a positive correction value as $c * t_{b_r}$ as in $ \rho = c * (t_r - t_s) + c * t_{b_r}$.

Each pseudo-range measurement gives a single constraint on receiver's state. Since we have 4 unknowns, $x, y, z, t_{b_r}$, so at least we need 4 measurements from 4 different satellites.

## Pseudo-range

### Code based pseudo range

The satellites broadcast two carrier waves constantly. These carrier waves
are in the L-Band (used for radio), and travel to earth at the speed of light.
These carrier waves are derived from the fundamental frequency, generated by a
very precise atomic clock: Frequency

- The L1 carrier is broadcast at 1575.42 MHz (10.23 x 154)
- The L2 carrier is broadcast at 1227.60 MHz (10.23 x 120).


The L1 carrier then has two codes modulated upon it. The C/A Code or Coarse/Acquisition Code is modulated at 1.023MHz (10.23/10) and has 1023 chips. The P-code or Precision Code is modulated at 10.23MHz. The L2 carrier has just one code modulated upon it. The L2 P-code is modulated at 10.23 MHz. C/A code is at a lower resolution ( 1 / 1.023Mhz = 977.5ns -> 293 meters) while P code is at a higher resolution (97.75ns -> 29.3 meter). However, this doesn't mean the pseudo range will have such large error. In reality, signal processing techniques can resolve fractions of a chip — even down to a few meters or less. But this does show that using P code is more accurate than using C/A code.


### Carrier-phase based pseudo range

Carrier phase tracking allows GPS receivers to use the high-frequency **carrier wave** to achieve **millimeter-level** distance measurements.


GPS receivers track the phase of the carrier signal, which is much finer than the pseudorandom code. The total distance measured this way is as below. Since wave length is very small (L1 is about 19cm and L2 is about 24cm), so it is very accurate. However, we need to figure out the integer N. 

\begin{align}
\rho_{\text{carrier}} = N \cdot \lambda + \phi \cdot \lambda
\end{align}

Where:
- $N$: Unknown integer number of full wavelengths (must be resolved)
- $\phi$ Fractional carrier phase (measured by receiver, in range $[0,1)$)
- $\lambda$ Wavelength of the GPS carrier


## GPS Error Sources

1. Ephemeris Errors (~2 meters)
2. Satellite Clock Errors (~2 meters)
3. Ionosphere Errors (~4 meters)
4. Troposphere Errors (~0.5 meters)
5. Multi-path Errors (~2 meters)
6. Dilution of Precision
5. Selective Availability (S/A)

## Double frequency receiver

The troposphere is the lower part of the atmosphere, thickest over the Equator (about 16 km). The temperature and pressure and humidity alter the speed of radio waves. Their effects are nearly independent of the radio frequency, but they depend on the time of passage. Ionospheric delay depends on frequency. So if the receiver can receives both L1 and L2, then we can get the real range $R = \frac{f_1^2 P_1 - f_2^2 P_2}{f_1^2 - f_2^2}$. This will removes the effects of   

\begin{align*}
P_1 &= R + \frac{K}{f_1^2} + \delta_{tropo}\\
P_2 &= R + \frac{K}{f_2^2} + \delta_{tropo}
\end{align*}

\begin{align*}
f_1^2 P_1 &= f_1^2 R + K + f_1^2 \delta_{tropo}\\
f_2^2 P_2 &= f_2^2 R + K + f_1^2 \delta_{tropo}
\end{align*}



## Differential GPS

- $ \rho_{A1} $: pseudorange from Receiver A to Satellite 1  
- $ \rho_{A2} $: pseudorange from Receiver A to Satellite 2  
- $ \rho_{B1} $: pseudorange from Receiver B to Satellite 1  
- $ \rho_{B2} $: pseudorange from Receiver B to Satellite 2  

$$
\rho_{ij} = R_{ij} + c \cdot \delta t_i - c \cdot \delta t_j
$$


- $ R_{ij} $: true range from receiver $ i $ to satellite $ j $
- $ \delta t_i $: receiver clock bias  
- $ \delta t_j $: satellite clock bias  
- $ c $: speed of light  


\begin{align}
\Delta_{AB,1} &= \rho_{A1} - \rho_{B1} = (R_{A1} - R_{B1}) + c(\delta t_A - \delta t_B) \\
\Delta\Delta &= \Delta_{AB,1} - \Delta_{AB,2} \\
&= [(R_{A1} - R_{B1}) - (R_{A2} - R_{B2})] + c[(\delta t_A - \delta t_B) - (\delta t_A - \delta t_B)] \\
&= (R_{A1} - R_{B1}) - (R_{A2} - R_{B2}) \\
\end{align}


The double-differenced measurement is shown as below. It removes satellite clock bias, receiver clock bias, and ionosphere and troposphere delays. If A is a reference station whose location is known, then we need 4 satellites to solve B's location.

$$
\Delta\Delta = (R_{A1} - R_{B1}) - (R_{A2} - R_{B2})
$$





## Receiver Position Estimation

Above shows different ways to get the pseudo ranges and errors sources for pseudo ranges. Now let's build an estimator to estimate the receiver position and clock bias. We model both of them as time variant variables. In addition, we will also model the clock drift as a time variant as well.

### Pseudo-range measurement

$$
\bar{\rho} = \hat{\rho} + c * t_{b_r} - c * t_{b_s} + \delta_{iono} + \delta_{tropo} + \delta_{sagnac}
$$

We need satellite navigation data from Rinex file ([readrnx](https://github.com/tomojitakasu/RTKLIB/blob/71db0ffa0d9735697c6adfd06fdf766d0e5ce807/src/rtklib.h#L1440C12-L1440C19), [uniqnav](https://github.com/tomojitakasu/RTKLIB/blob/71db0ffa0d9735697c6adfd06fdf766d0e5ce807/src/rtklib.h#L1351), [satposs](https://github.com/tomojitakasu/RTKLIB/blob/71db0ffa0d9735697c6adfd06fdf766d0e5ce807/src/rtklib.h#L1482C13-L1482C20)). [ionocorr](https://github.com/tomojitakasu/RTKLIB/blob/71db0ffa0d9735697c6adfd06fdf766d0e5ce807/src/rtklib.h#L1401) and [tropcorr](https://github.com/tomojitakasu/RTKLIB/blob/71db0ffa0d9735697c6adfd06fdf766d0e5ce807/src/rtklib.h#L1403C12-L1403C20) can be used to remove ionosphere and troposphere delay. 

One thing to notice is that we also need to consider Sagnac effect. The sagnac effect correction as in [geodist](https://github.com/tomojitakasu/RTKLIB/blob/71db0ffa0d9735697c6adfd06fdf766d0e5ce807/src/rtkcmn.c#L3199) is $\Omega * (x_{sat} * y_{rcv} - y_{sat} * x_{rcv}) / c$, where earth angular rage is $\Omega = 7.2921151467E-5 rad/s$ and light speed is $c = 299792458.0 m/s$



### Doppler measurement

Based on [resdop](https://github.com/tomojitakasu/RTKLIB/blob/71db0ffa0d9735697c6adfd06fdf766d0e5ce807/src/pntpos.c#L451), we have expected rate as below,
$$
v_{rel} = v_{sat} - v_{rcv} \\ 
l_{sight} = p_{sat} - p_{rcv} \\
\bar{d} = l_{sight}^T * v_{rel} + \Omega / c * (px_{rcv} * vy_{sat} + px_{sat} * vx_{rcv} - py_{rcv} * vx_{sat} - px_{sat} * vy_{rcv}) + t_{drift}
$$
where $\bar{d}$ is the doppler measurement in m/s, $t_{drift}$ is the clock drift rate in m/s.

 

### Carrier phase measurement

Suppose we have carrier range $r_t$ and $r_{t-1}$, then the carrier range difference $\delta_r = r_t - r_{t-1}$ (NOTE: if we have cycle slip, this might not be accurate).
$$
\rho_t = |(P_{sat, t} - P_{rcv, t})| + c * (t_{b_r}[t] - t_{b_c}) \\
\rho_{t-1} = |P_{sat, t-1} - P_{rcv, t-1}| + c * (t_{b_r}[t-1] - t_{b_c})
$$
Then 
$$
\delta_r = \rho_t - \rho_{t-1} + error
$$
$$
\delta_r =  |P_{sat, t} - P_{rcv, t}| - |P_{sat, t-1} + P_{rcv, t-1}| + c * (t_{b_r}[t] - t_{b_r}[t-1]) +  error
$$

### Process model

Use Eq. 14.88 of Principle of GNSS, Inertial, abd Multisensor Integrated Navigation System to get covariance of the process model.

$$
t_{b_r}[t] = t_{b_r}[t-1] + t_drift[t-1] * dt + error \\
t_drift[t] = t_drift[t-1] + error
$$